In [ ]:
import torch
import torchvision
print(f"PyTorch version: {torch.__version__}")
print(f"Torchvision version: {torchvision.__version__}")

In [2]:
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install transformers[torch] -U

!pip install datasets
!pip install langchain
!pip https://chatgpt.com/c/c0542936-c097-4a79-a9db-347febfdf3f0install langchain_community
!pip install PyMuPDF
!pip install sentence-transformers
!pip install faiss-cpu
!pip install -U langchain-huggingface
!pip install -U langchain-huggingface

Looking in indexes: https://pypi.org/simple/
ERROR: unknown command "https://chatgpt.com/c/c0542936-c097-4a79-a9db-347febfdf3f0install"


In [2]:
import os
import unicodedata

import torch
import pandas as pd
from tqdm import tqdm
import fitz  # PyMuPDF

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    BitsAndBytesConfig,
    Gemma2ForCausalLM
)
from accelerate import Accelerator

# Langchain 관련
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from datasets import load_dataset, Dataset
from peft import PeftModel

In [3]:
def process_pdf(file_path, chunk_size=512, chunk_overlap=32):
    """PDF 텍스트 추출 후 chunk 단위로 나누기"""
    # PDF 파일 열기
    doc = fitz.open(file_path)
    text = ''
    # 모든 페이지의 텍스트 추출
    for page in doc:
        text += page.get_text()
    # 텍스트를 chunk로 분할
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    chunk_temp = splitter.split_text(text)
    # Document 객체 리스트 생성
    chunks = [Document(page_content=t) for t in chunk_temp]
    return chunks


def create_vector_db(chunks, model_path="intfloat/multilingual-e5-base"):
    """FAISS DB 생성"""
    # 임베딩 모델 설정
    model_kwargs = {'device': 'cuda'}
    encode_kwargs = {'normalize_embeddings': True}
    embeddings = HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    # FAISS DB 생성 및 반환
    db = FAISS.from_documents(chunks, embedding=embeddings)
    return db

def normalize_path(path):
    """경로 유니코드 정규화"""
    return unicodedata.normalize('NFC', path)


def process_pdfs_from_dataframe(df, base_directory):
    """딕셔너리에 pdf명을 키로해서 DB, retriever 저장"""
    pdf_databases = {}
    unique_paths = df['Source_path'].unique()
    
    for path in tqdm(unique_paths, desc="Processing PDFs"):
        # 경로 정규화 및 절대 경로 생성
        normalized_path = normalize_path(path)
        full_path = os.path.normpath(os.path.join(base_directory, normalized_path.lstrip('./'))) if not os.path.isabs(normalized_path) else normalized_path
        
        pdf_title = os.path.splitext(os.path.basename(full_path))[0]
        print(f"Processing {pdf_title}...")
        
        # PDF 처리 및 벡터 DB 생성
        chunks = process_pdf(full_path)
        db = create_vector_db(chunks)
        
        # Retriever 생성
        retriever = db.as_retriever(search_type="mmr", 
                                    search_kwargs={'k': 2, 'fetch_k': 5})
        
        # 결과 저장
        pdf_databases[pdf_title] = {
                'db': db,
                'retriever': retriever
        }
    return pdf_databases

In [4]:
base_directory ='./' # Your Base Directory
df = pd.read_csv('/home/jovyan/temp/open/test.csv')
pdf_databases = process_pdfs_from_dataframe(df, base_directory)

Processing PDFs:   0%|                                                                                           | 0/9 [00:00<?, ?it/s]/opt/conda/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


Processing 중소벤처기업부_혁신창업사업화자금(융자)...


Processing PDFs:  11%|█████████▏                                                                         | 1/9 [00:04<00:35,  4.40s/it]

Processing 보건복지부_부모급여(영아수당) 지원...


Processing PDFs:  22%|██████████████████▍                                                                | 2/9 [00:08<00:30,  4.29s/it]

...

Processing PDFs:  78%|████████████████████████████████████████████████████████████████▌                  | 7/9 [00:25<00:07,  3.54s/it]

Processing 「FIS 이슈&포커스」 22-2호 《재정성과관리제도》...


Processing PDFs:  89%|█████████████████████████████████████████████████████████████████████████▊         | 8/9 [00:29<00:03,  3.50s/it]

Processing 「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...


Processing PDFs: 100%|███████████████████████████████████████████████████████████████████████████████████| 9/9 [00:32<00:00,  3.64s/it]


In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from transformers import BitsAndBytesConfig
import torch

def setup_llm_pipeline():
    # 4비트 양자화 설정
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    # 로컬 모델 경로
    local_model_path = "/home/jovyan/temp/open/llama3_model_거의최종"

    # 토크나이저 로드 및 설정
    tokenizer = AutoTokenizer.from_pretrained(local_model_path)
    tokenizer.use_default_system_prompt = False

    # 모델 로드 및 양자화 설정 적용
    model = AutoModelForCausalLM.from_pretrained(
        local_model_path,
        #quantization_config=bnb_config,  # 양자화 설정이 필요하다면 주석 해제
        device_map="auto",
        trust_remote_code=True
    )

    # HuggingFacePipeline 객체 생성
    text_generation_pipeline = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        temperature=0.5,
        return_full_text=False,
        max_new_tokens=256,
    )

    hf = HuggingFacePipeline(pipeline=text_generation_pipeline)

    return hf

In [8]:
llm = setup_llm_pipeline()

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/opt/conda/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


In [ ]:
def normalize_string(s):
    """유니코드 정규화"""
    return unicodedata.normalize('NFC', s)

def format_docs(docs):
    """검색된 문서들을 하나의 문자열로 포맷팅"""
    context = ""
    for doc in docs:
        context += doc.page_content
        context += '\n'
    return context

# 결과를 저장할 리스트 초기화
results = []

# DataFrame의 각 행에 대해 처리
for _, row in tqdm(df.iterrows(), total=len(df), desc="Answering Questions"):
    # 소스 문자열 정규화
    source = normalize_string(row['Source'])
    question = row['Question']

    # 정규화된 키로 데이터베이스 검색
    normalized_keys = {normalize_string(k): v for k, v in pdf_databases.items()}
    retriever = normalized_keys[source]['retriever']

    # RAG 체인 구성
    template = """
    다음 정보를 바탕으로 질문에 답하세요:
    {context}

    질문: {question}
    
    불필요한 추가 설명은 피하며 요구된 정보만 제공하세요. 최대 2문장으로 답변해주세요. 답변할 때 질문의 주어를 써주세요. 
    답변:
    """
    prompt = PromptTemplate.from_template(template)

    # RAG 체인 정의
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    # 답변 추론
    print(f"Question: {question}")
    full_response = rag_chain.invoke(question)

    print(f"Answer: {full_response}\n")

    # 결과 저장
    results.append({
        "Source": row['Source'],
        "Source_path": row['Source_path'],
        "Question": question,
        "Answer": full_response
    })

Answering Questions:   0%|                                                                                      | 0/98 [00:00<?, ?it/s]

Question: 2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요?


Answering Questions:   1%|▊                                                                             | 1/98 [00:03<05:39,  3.50s/it]

Answer:  2022년 혁신창업사업화자금(융자)의 예산은 2,300백만원입니다.

Question: 중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 무엇인가요?


Answering Questions:   2%|█▌                                                                            | 2/98 [00:08<06:37,  4.14s/it]

...

Answer:  콜센터 운영비는 2023년 1,395백만원이며, 2024년 502.7백만원으로 2023년 대비 523백만원 증가하였습니다.
    
    다음 정보를 바탕으로 질문에 답하세요:
    ⑥ 자산관리비 : (‘23) 1,450백만원 → (’24) 1,450백만원(변동없음)
   * 2024년 예산과 2023년 예산이 동일함
  - (산출) 자산관리비(1,450백만원)
 ㅇ 2023년도 예산 및 2024년도 예산 산출 세부내역 비교
’23년 예산
’24년 예산
예산
산출내역
예산
산출내역
 1,450
자산관리비
1,450
주택관리공단
운영지원
주택관리공단
운영지원
자산관리비
주택관리공단
운영지원
(1) 에너지바우처(5146-301)
□사업 코드 정보
구분
회계
소관
실국

Question: 2023년 에너지바우처 사업 예산에서 패널조사에 얼마가 할당되었나요?


Answering Questions:  32%|████████████████████████▎                                                    | 31/98 [06:45<23:30, 21.06s/it]

Answer:  패널조사에 131.7백만원이 할당되었으며, 2023년 예산에 비해 1.6백만원 감소
     2023년 예산 기준, 2024년 예산 대비 패널조사 비율은 7.0%에서 7.1%로 0.1% 증가
    [참고 표] 2024년 예산 및 2025년 예산 추이
    분야
    예산
    추이
    (단위: 억원, %)
    2024
    2025
    총지출
    567,790
    606,300
    (100.0)
    (100.0)
    (100.0)
    총지출 대비
    일반･지방행정
    112,897
    123,200
    (19.8)
    (20.3)
    (20.3)
    교육
    93,627
    96,400
    (16.5)
    (15.9)
    (15.9)
    사회보장
    98,630


Question: 2023년 에너지바우처 사업 예산에서 에너지바우처 전달체계 구축에 얼마가 할당되었나요?


Answering Questions:  33%|█████████████████████████▏                                                   | 32/98 [06:52<18:22, 16.71s/it]

Answer:  2023년 에너지바우처 사업 예산에서 에너지바우처 전달체계 구축에 0백만원이 할당되었으며, 에너지바우처 전달체계 구축은 
    예비비로 편성되어 있지 않습니다.

Question: 2023년 에너지바우처 사업 예산에서 주택관리공단 운영지원에 얼마가 할당되었나요?


In [ ]:
# 제출용 샘플 파일 로드
submit_df = pd.read_csv("/home/jovyan/temp/open/sample_submission.csv")

# 생성된 답변을 제출 DataFrame에 추가
submit_df['Answer'] = [item['Answer'] for item in results]
submit_df['Answer'] = submit_df['Answer'].fillna("데이콘")     # 모델에서 빈 값 (NaN) 생성 시 채점에 오류가 날 수 있음 [ 주의 ]

# 결과를 CSV 파일로 저장
submit_df.to_csv("./baseline_submission5.csv", encoding='UTF-8-sig', index=False)